High Value Customer Identification (Insiders)

In [120]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import requests

In [48]:
!pip list --format=freeze > requirements.txt

# Multiple products

In [124]:
url = 'https://www2.hm.com/en_us/men/products/jeans.html'

headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text, "html.parser")

total_item = soup.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )
total_item

'90'

In [125]:
page_number = np.round( int( total_item ) / 36 )
page_number

2.0

In [147]:
urlfull = url + '?page-size=' + str( int( page_number*36 ) )
urlfull

'https://www2.hm.com/en_us/men/products/jeans.html?page-size=72'

In [152]:
page = requests.get( urlfull , headers=headers )


soup = BeautifulSoup( page.text, "html.parser")

#total_item = p.find_all( 'h2', class_='load-more-heading' )[0].get( 'data-total' )

total_item = [p.get("data-articlecode") for p in soup.find_all("article", class_='hm-product-item')]

total_item[:3]

['1024256001', '0985159001', '1008549001']

In [203]:
df_details = pd.DataFrame()

#unique columns for all products
products_ids = []

cols = ['Additional material information','Art. No.','Care instructions','Collection','Composition','Concept','Description','Fit','Imported','Material','Size']
df_pattern = pd.DataFrame( columns = cols)

for i in total_item:

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    aux = soup.find_all('a', class_='filter-option miniature active') + soup.find_all('a', class_='filter-option miniature')
    aux = [p.get('data-articlecode') for p in aux]
    products_ids.append(aux)

In [206]:
products_ids2 = []

for i in products_ids:
    products_ids2 += i

products_ids2 = list(set(products_ids2))

products_ids2

['1013317004',
 '1004199004',
 '0811993039',
 '0938875012',
 '0690449040',
 '0875105026',
 '0690449009',
 '1024256003',
 '0993887004',
 '0938875001',
 '0690449067',
 '0690449002',
 '1024256007',
 '0690449064',
 '0811993024',
 '1008549002',
 '0979945019',
 '0979945009',
 '1008549004',
 '0979945012',
 '1013317006',
 '0985159006',
 '0811993037',
 '0979945002',
 '0811993003',
 '0811993044',
 '0938875010',
 '0811993025',
 '1114023001',
 '0875105017',
 '1013317001',
 '0985159005',
 '0985159002',
 '1004199002',
 '0690449036',
 '1008110019',
 '1071707003',
 '0811993001',
 '0811993028',
 '0811993007',
 '1024711004',
 '1008549006',
 '0811993036',
 '1096385002',
 '1107750001',
 '1008549003',
 '1107750002',
 '1004199003',
 '1024711008',
 '0690449028',
 '0875105003',
 '1071707002',
 '0690449024',
 '1008110002',
 '0690449051',
 '0938875007',
 '0875105009',
 '1008549005',
 '0875105042',
 '0690449001',
 '0938875014',
 '1024256008',
 '0690449059',
 '1024256006',
 '0811993006',
 '1100162002',
 '09388750

In [270]:
cols = [ 'product_name', 'price', 'composition', 'color', 'size', 'fit', 'article_code']
df_pattern = pd.DataFrame( columns = cols)



url = f'https://www2.hm.com/en_us/productpage.1013317003.html'

page = requests.get( url, headers=headers )

soup = BeautifulSoup( page.text , 'html.parser' )

name_price = soup.find_all('section', class_='product-name-price')[0]

product_name = name_price.find('h1').text
product_price = name_price.find('span', class_='price-value').text

In [271]:
product_price

'\r\n                $39.99'

In [207]:

df_details = pd.DataFrame()

#unique columns for all products
aux = []

cols = [ 'product-name', 'composition', 'color', 'size', 'fit', 'article-code']
df_pattern = pd.DataFrame( columns = cols)

for i in products_ids2[:2]:

    url = f'https://www2.hm.com/en_us/productpage.{i}.html'

    page = requests.get( url, headers=headers )

    soup = BeautifulSoup( page.text , 'html.parser' )

    product_list = soup.find_all('div', class_='details-attributes-list-item')


    # remove empty strings
    product_composition = [list(filter(None, p.get_text().split('\n'))) for p in product_list]


    # list to dataframe
    data_composition = pd.DataFrame(product_composition).T


    # data aux
    data_composition_aux = data_composition.iloc[0]


    # rename df columns
    data_composition.columns = data_composition_aux

    aux = aux + data_composition.columns.tolist()

    aux = [i for i in aux if "messages." not in i]

    # forward fillna
    data_composition = data_composition.iloc[1:].fillna(method='ffill')

    data_composition['Composition'] = data_composition['Composition'].str.replace( "Shell: |Pocket lining: |Lining","", regex=True )

    data_composition = pd.concat( [df_pattern, data_composition], axis=0)


    # drop duplicates
    data_composition = data_composition.drop_duplicates(subset=['Fit', 'Composition', 'Art. No.'], keep='first', inplace=False, ignore_index=False)


    # only selected columns
    data_composition = data_composition[cols].copy()

    data_composition.columns = data_composition.columns.str.lower()

    data_composition['style_id'] = data_composition['art. no.'].str[:-3]

    data_composition['color_id'] = data_composition['art. no.'].str[-3:]



    # Produto Final
    df_details = pd.concat( [df_details, data_composition] , axis=0)

KeyError: 'art. no.'

In [58]:
df_products2.to_csv("df_products.csv", index=False)